In [2]:
import pandas as pd
import os
import requests as req
from requests import get
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
import time
from time import sleep 
from selenium.webdriver.common.keys import Keys

import urllib
import requests
from bs4 import BeautifulSoup

import os
import glob

path = 'C:/Users/Tonyr/downloads'
os.chdir(path)
cwd = os.getcwd()
sd = glob.glob('transactions.csv')

if len(sd) > 0:
    os.remove(sd[0])
    
    



req = req.get('https://mint.com')

input('Enter Email Id:')  
input('Enter Password:') 

url = "https://www.mint.com/"
path = 'C:/Users/tonyr/ChromeDriver/chromedriver_win32'
os.chdir(path)
cwd = os.getcwd()
options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument('--ignore-certificate-errors')
options.add_argument("--test-type")
# options.add_argument("headless")
driver = webdriver.Chrome(options=options)


driver.get(url)
print ("Opened Mint") 
sleep(1) 

Login = driver.find_element_by_xpath('/html/body/div[1]/div/section[1]/header/div/div[3]/a[2]')
Login.click()

username_box = driver.find_element_by_id('ius-userid') 
username_box.send_keys(usr) 
print ("Email Id entered") 
sleep(1) 
  
password_box = driver.find_element_by_id('ius-password') 
password_box.send_keys(pwd) 
print ("Password entered") 

SignInClick = driver.find_element_by_id('ius-sign-in-submit-btn-text') 
SignInClick.click() 
print ("Signed In") 
sleep(30)

x = driver.find_element_by_class_name('pageContentSection')
xtext = x.text
while xtext.lower().find('refreshing') >= 0:
    print('Waiting For Refresh')
    sleep(10)
    x = driver.find_element_by_class_name('pageContentSection')
    xtext = x.text
transactions = driver.find_element_by_id('transaction')
transactions.click()
sleep(30)

transactionsexport = driver.find_element_by_id('transactionExport')
driver.execute_script("arguments[0].click();", transactionsexport)




In [3]:
path = 'C:/Users/Tonyr/'
os.chdir(path)
cwd = os.getcwd()

filenew = 'Downloads/transactions.csv'
fileold = 'Desktop/Self Education/production files/transactions.csv'



transactionsnew = pd.read_csv(filenew,parse_dates = [0])
transactionsold = pd.read_csv(fileold, parse_dates = [0])
transactionsold['date'] = pd.to_datetime(transactionsold['date'])
transactionsnew['Date'] = pd.to_datetime(transactionsnew['Date'])




In [4]:
import numpy as np 
transactionsnew

conditions = [(transactionsnew['Transaction Type'] == 'debit'),(transactionsnew['Transaction Type'] == 'credit')]
choices = [transactionsnew['Amount'] * - 1 , transactionsnew['Amount']]
transactionsnew['Amount Transformed'] = np.select(conditions, choices, default = 'Fucker')


print(transactionsold['date'].min(),' ',  transactionsold['date'].max())

print(transactionsnew['Date'].min(),' ',  transactionsnew['Date'].max())

2016-12-30 00:00:00   2019-03-23 00:00:00
2016-12-30 00:00:00   2019-03-24 00:00:00


## Only Grab Data between Prior Thursday 10/11 days agao when done on Saturday/Sunday thru previous Wednesday 3/4 days

In [5]:
import datetime 
today = datetime.datetime.now()
end = datetime.timedelta(days = 0)
start = datetime.timedelta(days = 14)

end2 = today - end
end2 = end2.replace(hour = 0, minute = 0, second = 0,microsecond = 0)
end2

start2 = today - start
start2 = start2.replace(hour = 0, minute = 0, second = 0,microsecond = 0)


print('Start2:',start2,'End2:', end2)


x = transactionsold['date'].max()
New = transactionsnew[(transactionsnew['Date'] >= start2) & (transactionsnew['Date'] <=end2) ]
Old = transactionsold[transactionsold['date'] < start2]

print('Range Bringing In:', New['Date'].min(),' ',  New['Date'].max())

print('Range Keeping:    ' , Old['date'].min(),' ',  Old['date'].max())

New = New[['Date','Description', 'Original Description', 'Amount', 'Amount Transformed', 'Transaction Type' , 'Category'
     , 'Account Name', 'Labels']]


New.columns = Old.columns

Start2: 2019-03-11 00:00:00 End2: 2019-03-25 00:00:00
Range Bringing In: 2019-03-11 00:00:00   2019-03-24 00:00:00
Range Keeping:     2016-12-30 00:00:00   2019-03-10 00:00:00


In [6]:

from sqlalchemy import create_engine

transactionsupdate = pd.concat([New, Old])
cols = ['date', 'description', 'originaldescription', 'amount','amount_transformed', 'transaction_type' , 'category','account_name', 'sub_cat']
transactionsupdate.columns = cols

engine = create_engine('postgresql+psycopg2://postgres:<Password>@localhost:<Port>/<Database>')


transactionsupdate.to_sql('transactionsupdate' , con = engine, if_exists = 'replace')

In [12]:
import pandas
from pandas import DataFrame

connection = engine.connect()
execute = connection.execute("Select * from transactionsupdate")
DF = pandas.DataFrame(execute.fetchall())
DF.columns = execute.keys()

del DF['index']


DF.to_csv('C:/Users/Tonyr/Desktop/Self Education/production files/transactions.csv', index = False)